```
This software is a part of GPU Ocean.

Copyright (C) 2019  SINTEF Digital

This notebook reads the data from the rank histogram experiments and
creates the rank histogram.

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

# Rank histogram


This notebook reads the data from the rank histogram experiments and
creates the rank histogram.


## Set environment

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import animation, rc

import os
import sys
import datetime

from importlib import reload
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../../')))

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')

#Import our simulator
from SWESimulators import IPythonMagic, SimReader, Observation
from SWESimulators import DoubleJetCase
from SWESimulators import DataAssimilationUtils as dautils

In [ ]:
%cuda_context_handler gpu_ctx
timestamp = datetime.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")

### Define functions for plotting

In [ ]:
def imshow3(eta, hu, hv, interpolation="None", title=None, figsize=(12,3), interior=False):
    fig, axs = plt.subplots(1,3, figsize=figsize)
        
    range_eta = np.max(np.abs(eta))
    range_huv = max(np.max(np.abs(hu)), np.max(np.abs(hv)))
    
    eta_im = axs[0].imshow(eta, interpolation=interpolation, origin='lower', vmin=-range_eta, vmax=range_eta)
    axs[0].set_title("$\eta$")
    plt.colorbar(eta_im, ax=axs[0])
    
    if interior:
        hu_im = axs[1].imshow(hu[2:-2,2:-2], interpolation=interpolation, origin='lower', vmin=-range_huv, vmax=range_huv)
    else:
        hu_im = axs[1].imshow(hu, interpolation=interpolation, origin='lower', vmin=-range_huv, vmax=range_huv)
    axs[1].set_title("$hu$")
    plt.colorbar(hu_im, ax=axs[1])

    if interior:
        hv_im = axs[2].imshow(hv[2:-2,2:-2], interpolation=interpolation, origin='lower', vmin=-range_huv, vmax=range_huv)
    else:
        hv_im = axs[2].imshow(hv, interpolation=interpolation, origin='lower', vmin=-range_huv, vmax=range_huv)
    axs[2].set_title("$hv$")
    plt.colorbar(hv_im, ax=axs[2])

    if title is not None:
        plt.suptitle(title)
    plt.tight_layout()
    
def plotDrifters(observations, sim_reader, t, ax=None):
    drifter_positions = observations.get_drifter_position(t)

    nx, ny = sim_reader.get('nx'), sim_reader.get('ny')
    dx, dy = sim_reader.get('dx'), sim_reader.get('dy')

    if ax is None:
        fig = plt.figure(figsize=(7,7))
        ax = plt.subplot(111)
        emptyData =np.ma.masked_where(np.zeros((ny,nx)) > 1, np.zeros((ny,nx)))
        ax.imshow(emptyData, origin="lower", extent=[0, nx*dx, 0, ny*dy], cmap='binary')

    for i in range(drifter_positions.shape[0]):
        color = 'xkcd:tomato red'
        circ_end = matplotlib.patches.Circle((drifter_positions[i,0], drifter_positions[i,1]),
                                             3000, fill=True, zorder=10, color=color)
        ax.add_patch(circ_end)

    
def days_to_sec(days):
    return days*24*60*60

def truth_time_step(t):
    t = t - days_to_sec(3)
    return int(t/(60*60))

def file_filter(path_to_dir, ext=None, prefix=None, abspath=True):
    filtered_files = os.listdir(path_to_dir)
    if prefix:
        filtered_files = list(file for file in filtered_files if file.startswith(prefix))
    if ext:
        filtered_files = list(file for file in filtered_files if file.endswith(ext))
    if abspath:
        filtered_files= list(os.path.join(path_to_dir, file)  for file in filtered_files)

    filtered_files.sort()
    return filtered_files

# Reading the truth

Loading the truth to see which parts of the domain that can be relevant. 

We use buoys in order to assimilate the same data points for each round.

In [ ]:
truth_path = os.path.abspath('scripts/truth_2019_06_06-09_23_41/')
assert len(os.listdir(truth_path)) == 4, "Truth folder has wrong number of files"

media_dir = '/media/havahol/Seagate Backup Plus Drive/gpu_ocean'

In [ ]:
truth_state_filename = os.path.join(truth_path, "double_jet_case_truth.nc")
observations_filename = os.path.join(truth_path, "drifter_observations.pickle")

truth_reader = SimReader.SimNetCDFReader(truth_state_filename)

observation_type = dautils.ObservationType.StaticBuoys
nx = truth_reader.get('nx')
ny = truth_reader.get('ny')
dx = truth_reader.get('dx')
dy = truth_reader.get('dy')
domain_size_x = nx*dx
domain_size_y = ny*dy

observations = Observation.Observation(observation_type=observation_type,
                                       domain_size_x=domain_size_x, domain_size_y=domain_size_y,
                                       nx=nx, ny=ny)
observations.read_pickle(observations_filename)
observations.setBuoyReadingArea('west')

x_index = 100


In [ ]:
# Get files
#rank_histogram_paths = [
#    os.path.join(media_dir, 'june_truth/rank_histogram_1_hour/rank_histogram_experiments_2019_06_06-16_40_27/')
#]

#dataset = 'june_truth/rank_histogram_72_hour/'
#dataset = 'june_truth/rank_histogram_forecast_6_1_hour/'
#dataset = 'june_truth/rank_histogram_curand_forecast_6_1_hour/'
dataset = 'unique_truths/rank_histogram_tests/'
dataset_path = os.path.join(media_dir, dataset)

fig_path = os.path.join(dataset_path, 'figures_' + timestamp)
if not os.path.exists(fig_path):
    os.mkdir(fig_path)
    print('created figure folder')
    print(fig_path)

rank_histogram_time = 6

rank_histogram_paths = list(os.path.join(dataset_path, path) for path in os.listdir(dataset_path))
print(rank_histogram_paths)

rank_histogram_files = []
for path in rank_histogram_paths:
    rank_histogram_files.extend(file_filter(path, 
                                            ext='npz', 
                                            prefix='hour_' + str(rank_histogram_time).zfill(3)))

print(len(rank_histogram_files))

In [ ]:
# Read the first obtained values
first_experiment = np.load(rank_histogram_files[0])
t_rank = float(first_experiment['t'])
print(t_rank)


In [ ]:
t_rank_index = truth_time_step(t_rank)
eta_rank, hu_rank, hv_rank, t = truth_reader.getTimeStep(t_rank_index)
print(t, t_rank_index)
imshow3(eta_rank, hu_rank, hv_rank, title='State at rank histogram time')

buoy_cell_indices = observations.buoy_indices[observations.read_buoy, :]
print(buoy_cell_indices.shape)

buoy_y = np.zeros(12)
buoy_x = 100
bouy_y_map = [False]*ny

fig = plt.figure(figsize=(7,7))

ax = plt.subplot(111)
ax.imshow(hu_rank, origin="lower", extent=[0, nx, 0, ny])

buoy_index = 0
for i in range(buoy_cell_indices.shape[0]):
    color = 'xkcd:tomato red'
    if buoy_cell_indices[i,0] == buoy_x:
        print(buoy_cell_indices[i,:])
        color = 'xkcd:white'
        buoy_y[buoy_index] = buoy_cell_indices[i,1]
        buoy_index += 1
        bouy_y_map[buoy_cell_indices[i,1]] = True
    circ_end = matplotlib.patches.Circle((buoy_cell_indices[i,0], buoy_cell_indices[i,1]),
                                         2, fill=True, zorder=10, color=color)
    ax.add_patch(circ_end)
    
    
plt.title('at rank hist time, with buoys')
plt.savefig(os.path.join(fig_path, 'truth_with_buoys.png'))
plt.savefig(os.path.join(fig_path, 'truth_with_buoys.pdf'))

In [ ]:
observed_values = observations.get_observation(t_rank, None)

rounded_t = round(t_rank)
t_rank_index_obs = observations.obs_df[observations.obs_df[observations.columns[0]]==rounded_t].index.values[0]

print(rounded_t)
print('type of observations.buoy_indices: ', observations.buoy_indices.dtype)

print('direct positions (type ' + str(observations.buoy_positions.copy()[observations.read_buoy, :].dtype) + ')')
#print(observations.buoy_positions.copy()[observations.read_buoy, :])
#print('direct hu hv:')
#print(observations.obs_df.iloc[index][observations.columns[2]][observations.read_buoy, :])

eta_rank, hu_rank, hv_rank, t = truth_reader.getTimeStep(t_rank_index)
print(t, t_rank_index)
eta_true = eta_rank[:, buoy_x]
hu_true = hu_rank[:, buoy_x]
hv_true = hv_rank[:, buoy_x]

if 'true_hu' in first_experiment.files:
    eta_true = first_experiment['true_eta']
    hu_true = first_experiment['true_hu']
    hv_true = first_experiment['true_hv']

print(observed_values.shape)
print(observed_values[4::10, 2])
print(observed_values[4::10, 2]-hu_rank[bouy_y_map, buoy_x])

# Plotting the cross section at the chosen buoy locations:
fig = plt.figure(figsize=(12,4))
fig, axs = plt.subplots(3, 1, figsize=(12,12))


axs[0].plot(np.arange(ny), hu_true, 'k', zorder=10)
first_hu = first_experiment['hu']
for p in range(first_hu.shape[1]):
    axs[0].plot(np.arange(ny), first_hu[:,p], color='xkcd:light blue', alpha=0.2)
axs[0].plot(buoy_y, hu_true[bouy_y_map], 'r*')
axs[0].set_title('ensemble for eta at x_100')


fig = plt.figure(figsize=(12,4))
first_hv = first_experiment['hv']
for p in range(first_hu.shape[1]):
    axs[1].plot(np.arange(ny), first_hv[:,p], color='xkcd:light blue', alpha=0.2)
axs[1].plot(np.arange(ny), hv_true, 'k')
axs[1].plot(buoy_y, hv_true[bouy_y_map], 'r*')
axs[1].set_title('ensemble for hv at x_100')


fig = plt.figure(figsize=(12,4))
first_eta = first_experiment['eta']
for p in range(first_hu.shape[1]):
    axs[2].plot(np.arange(ny), first_eta[:,p], color='xkcd:light blue', alpha=0.2)
axs[2].plot(np.arange(ny), eta_true, 'k')
axs[2].plot(buoy_y, eta_true[bouy_y_map], 'r*')
axs[2].set_title('ensemble for eta at x_100')

plt.savefig(os.path.join(fig_path, 'ensemble_cross_section_x_100.png'))
plt.savefig(os.path.join(fig_path, 'ensemble_cross_section_x_100.pdf'))


# Rank histogram experiment

Rank histograms are an established way to test the quality of data assimilation methods (see reference Hamill 2001 from Skauvold et al). Our rank histogram experiment is based on the buoy experiment in which we observe the western half of the domain with $R = \mathrm{diag}(1)$ and assimilation windows of five minutes.  The ensemble size is sat to $N_e = 40$. For each experiment we store the cross section at $x = 100.5 \Delta x$ (center of the 100'the cell in $x$-direction).

In [ ]:
def rankhistogram(dataset_path, hour, truth_reader, interesting_y_indices, perturb=True):
    

    rank_histogram_paths = list(os.path.join(dataset_path, path) for path in os.listdir(dataset_path))
    #print(rank_histogram_paths)

    rank_histogram_files = []
    for path in rank_histogram_paths:
        rank_histogram_files.extend(file_filter(path, 
                                                ext='npz', 
                                                prefix='hour_' + str(hour).zfill(3)))

    print('Creating rank histogram from ' + str(len(rank_histogram_files)) + ' files')
    
    # Time in seconds is read from first file
    first_experiment = np.load(rank_histogram_files[0])
    t_rank = float(first_experiment['t'])
    
    # Index in the truth corresponding to the provided time
    t_rank_index = truth_time_step(t_rank)
    
    
    num_rank_experiments = len(rank_histogram_files)
    Ne = 40
    buoy_x = 100 # Index of buoy

    true_eta, true_hu, true_hv, true_t = truth_reader.getTimeStep(t_rank_index)
    ny = truth_reader.get('ny')

    buckets_hu  = np.zeros((ny, Ne+1))
    buckets_hv  = np.zeros((ny, Ne+1))
    buckets_eta = np.zeros((ny, Ne+1))

    interesting_y_indices = interesting_y_indices.astype(np.int32) # [0, 25, 50, 70, 75, 80, 100, 125, 150, 175]
    
    common_true_eta = true_eta[:,buoy_x]
    common_true_hu  = true_hu[:, buoy_x]
    common_true_hv  = true_hv[:, buoy_x]

    exp_no = 0
    for file in rank_histogram_files:
        result = np.load(file)
        eta = result['eta'] 
        hu  = result['hu']
        hv  = result['hv'] 
        t = result['t']
        assert(true_t == t)
        
        true_eta, true_hu, true_hv = None, None, None
        if 'true_eta' in result.files:
            true_eta = result['true_eta']
            true_hu  = result['true_hu']
            true_hv  = result['true_hv']
        else:
            true_eta = common_true_eta
            true_hu  = common_true_hu
            true_hv  = common_true_hv

        for y in interesting_y_indices:
            if perturb:
                ensemble_hu  = hu[y, :] + np.random.normal(scale=1, size=Ne)
                ensemble_hv  = hv[y, :] + np.random.normal(scale=1, size=Ne)
            else:
                ensemble_hu  = hu[y, :] 
                ensemble_hv  = hv[y, :] 
            ensemble_eta = eta[y, :] 

            ensemble_hu.sort()
            ensemble_hv.sort()
            ensemble_eta.sort()

            bucket_hu  = np.searchsorted(ensemble_hu, true_hu[y])
            bucket_hv  = np.searchsorted(ensemble_hv, true_hv[y])
            bucket_eta = np.searchsorted(ensemble_eta, true_eta[y])

            buckets_hu[y, bucket_hu] += 1
            buckets_hv[y, bucket_hv] += 1
            buckets_eta[y, bucket_eta] += 1

        if num_rank_experiments > 99:
            if (exp_no % int(num_rank_experiments/100) == 0):
                print('.'+str(exp_no)+'.', end='')
        exp_no += 1

    accumulated_eta = np.zeros_like(buckets_eta[0,:])
    accumulated_hu  = np.zeros_like(buckets_hu[0,:])
    accumulated_hv  = np.zeros_like(buckets_hv[0,:])
    
    # Plot results
    for y in interesting_y_indices:
        
        accumulated_eta += buckets_eta[y,:]
        accumulated_hu  += buckets_hu[y,:]
        accumulated_hv  += buckets_hv[y,:]
        
        fig, axs = plt.subplots(1, 3, figsize=(15,3))

        axs[0].bar(np.arange(Ne+1), buckets_eta[y,:], width=1)
        axs[0].set_title('eta at cell (100, '+str(y)+')')

        axs[1].bar(np.arange(Ne+1), buckets_hu[y,:], width=1)
        axs[1].set_title('hu at cell (100, '+str(y)+')')

        axs[2].bar(np.arange(Ne+1), buckets_hv[y,:], width=1)
        axs[2].set_title('hv at cell (100, '+str(y)+')')
        
        plt.suptitle('Rank histogram at time hour ' + str(hour), fontsize=15)
        #plt.tight_layout()
        fig.tight_layout(rect=[0, 0.03, 1, 0.95])
    
        filename = 'rank_histogram_y_'+str(y).zfill(3)+'_hour_'+str(hour).zfill(3)+'.pdf'
        plt.savefig(os.path.join(fig_path, filename))
        plt.savefig(os.path.join(fig_path, filename).replace('.pdf', '.png'))
    
    fig, axs = plt.subplots(1, 3, figsize=(15,3))

    axs[0].bar(np.arange(Ne+1), accumulated_eta, width=1)
    axs[0].set_title('accumulated eta')

    axs[1].bar(np.arange(Ne+1), accumulated_hu, width=1)
    axs[1].set_title('accumulated hu')

    axs[2].bar(np.arange(Ne+1), accumulated_hv, width=1)
    axs[2].set_title('accumulated hv')

    plt.suptitle('Accumulated rank histogram at time hour ' + str(hour), fontsize=15)
    #plt.tight_layout()
    fig.tight_layout(rect=[0, 0.03, 1, 0.95])

    str_interesting_y_indices= str(interesting_y_indices).replace('[  ','').replace(']','').replace('  ', ' ').replace(' ', '_')
    filename = 'rank_histogram_accumulated_'+str_interesting_y_indices+'_hour_'+str(hour).zfill(3)+'.pdf'
    plt.savefig(os.path.join(fig_path, filename))
    plt.savefig(os.path.join(fig_path, filename).replace('.pdf', '.png'))

In [ ]:
hour = 7
#interesting_y_indices = np.array([0, 25, 50, 70, 75, 80, 100, 125, 150, 175])
interesting_y_indices = np.array([0, 50, 100, 150, 200, 250])
print(dataset_path)
rankhistogram(dataset_path, hour, truth_reader, interesting_y_indices, perturb=True)  


In [ ]:
hour = 7
interesting_y_indices = np.array([0, 37, 50])
rankhistogram(dataset_path, hour, truth_reader, interesting_y_indices, perturb=True)  


# Time development of rank histogram

In [ ]:

hours = np.array([1, 6, 12, 18, 24, 48, 72])
y = np.array([200])

for hour in hours:
    rankhistogram(dataset_path, hour, truth_reader, y, perturb=False)  




In [ ]:
files = np.arange(10)
print(files)
np.random.choice(files, size=7, replace=False)

In [ ]:
ensemble_directory = 'double_jet_ensemble_init/'
ensemble_init_nc_files = list(os.path.join(ensemble_directory, file) for file in os.listdir(ensemble_directory) if file.endswith('.nc'))
print(len(ensemble_init_nc_files))
np.random.choice(ensemble_init_nc_files, size=10, replace=False)

In [ ]:
npload = np.load(rank_histogram_files[0])
print(npload.files)
if 'true_hu' in npload.files:
    print('hei')

In [ ]:
print('time (in days) for 1000 rank histogram experiments')
print('40 west buoys', 37.5*24*1000/(60*60*24))
print('40 10-drifters', 19*24*1000/(60*60*24))
print('25 west buoys', 23*24*1000/(60*60*24))

In [ ]:
670 / 60, 37.5*24

In [ ]:
exp_24h = 670
exp_48h = exp_24h*2

num_days_100_exp_24 = exp_24h*100/(60*60*24)
print("100 experiments at 24 hours: ", num_days_100_exp_24)


time_until_monday_at_work = (8 + 24 + 24 + 24 + 8)*3600
num_24_hour_experiments_until_monday = time_until_monday_at_work/exp_24h
print('num_24_hour_experiments_until_monday', num_24_hour_experiments_until_monday)
num_48_hour_experiments_until_monday = time_until_monday_at_work/exp_48h
print('num_48_hour_experiments_until_monday', num_48_hour_experiments_until_monday)
